# Snow-cover attributes and time-series extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the snow-cover time-series from the MODIS dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Observations
* This notebook assumes that the GEE code to export snow-cover mean time-series from the MODIS dataset were run before in the GEE platform and that the output CSV-files are locally available. 
* It is not possible to export the 15,000 catchments at one single CSV, so there might be many files with the time-series stored separetly. 

## Requirements
**Python:**
* Python>=3.6
* Jupyter
* geopandas=0.10.2
* glob
* numpy
* os
* pandas
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**
* data/shapefiles/estreams_catchments.shp
* data/gee/snowcover/EStreams_modis_snow_cover_mean_gee_{}.csv. Snow-cover time-series CSV-file(s) exported from GEE

**Directory:**
* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Hall, D. K., V. V. Salomonson, and G. A. Riggs. 2016. MODIS/Terra Snow Cover Daily L3 Global 500m Grid. Version 6. Boulder, Colorado USA: NASA National Snow and Ice Data Center Distributed Active Archive Center.

## Licenses

* Snow Cover: Open access: You may download and use photographs, imagery, or text from the NSIDC web site, unless limitations for its use are specifically stated. For more information on usage and citing NSIDC datasets, please visit the [NSIDC 'Use and Copyright' page] (https://nsidc.org/about/data-use-and-copyright).(Last access 27 November 2023) 

## Observations
* This notebook assumes that the GEE code to export snow cover time-series from the MODIS dataset (EStreams_landscape_timeseries_Snow_Cover_gee.txt) was run before in the GEE platform and that the output CSV-files are locally available. 

# Import modules

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tqdm as tqdm
import glob

# Configurations

In [2]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.

In [3]:
# Non-editable variables:
PATH_OUTPUT_TS = "results/timeseries/snowcover"
PATH_OUTPUT_ST = "results/staticattributes"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [4]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries

,id,area_km2,outlet_lat,outlet_lng,name,area_offic,layer,path,Code,basin_id,area_calc,geometry
0,FR003159,37,47.488,7.393,A100003001,38.6,FR003159,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003159,FR003159,37.183,"POLYGON ((7.30374 47.49375, 7.30708 47.49375, ..."
1,FR003160,227,47.626,7.239,A105003001,233.0,FR003160,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003160,FR003160,226.962,"POLYGON ((7.22291 47.63458, 7.22374 47.63458, ..."
2,FR003161,14,47.586,7.384,A106000101,15.0,FR003161,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003161,FR003161,13.595,"POLYGON ((7.38791 47.59041, 7.39874 47.59041, ..."
3,FR003162,70,47.622,7.275,A107020001,70.0,FR003162,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003162,FR003162,70.152,"POLYGON ((7.28375 47.60958, 7.28291 47.60958, ..."
4,FR003163,330,47.653,7.265,A108003001,325.0,FR003163,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003163,FR003163,330.158,"POLYGON ((7.22958 47.65291, 7.23208 47.65291, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1967,HR000314,135,44.202,16.069,7267,NaN,HR000314,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,HR000314,HR000314,135.462,"POLYGON ((16.01458 44.21375, 16.01375 44.21375..."
1968,HR000315,458,44.162,15.858,7236,NaN,HR000315,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,HR000315,HR000315,457.864,"POLYGON ((15.89625 44.07791, 15.89374 44.07791..."
1969,HR000316,514,44.162,15.849,7237,NaN,HR000316,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,HR000316,HR000316,514.369,"POLYGON ((15.84208 44.15458, 15.84208 44.15458..."
1970,HR000317,185,45.334,14.452,6077,NaN,HR000317,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,HR000317,HR000317,184.733,"POLYGON ((14.51875 45.36708, 14.51875 45.36791..."


## GEE outputs
### Snow-cover

In [5]:
# Check the files in the subdirectory:
filenames = glob.glob("data/gee/snowcover/*.csv")
print("Number of files:", len(filenames))
print("First file:", filenames[0])

Number of files: 82
First file: data/gee/snowcover/EStreams_modis_snow_cover_mean_gee_1425_1449.csv


In [6]:
# First, we create an empty DataFrame for the data with a datetime index:
snowcover_df = pd.DataFrame(index=pd.date_range(start='2001-01-01', end='2022-12-31', freq='M'))

# Loop for reading and concatenating the data:
for file in tqdm.tqdm(filenames):
    
    # Read the data from the CSV file:
    snowcover_file = pd.read_csv(file)
    snowcover_file.drop(["system:index", ".geo"], axis=1, inplace=True)
    snowcover_file = snowcover_file.T
    
    # Set columns based on the "basin_id" row and drop it
    snowcover_file.columns = snowcover_file.loc["basin_id", :].tolist()
    snowcover_file.drop(["basin_id"], axis=0, inplace=True)
    
    # Convert the index to integers and sort it
    snowcover_file.index = snowcover_file.index.astype(int)
    snowcover_file.sort_index(inplace=True)
    
    # Create a new DataFrame with datetime index and assign values
    snowcover_file_df = pd.DataFrame(columns=snowcover_file.columns)
    snowcover_file_df["dates"] = pd.date_range(start='2001-01-01', end='2022-12-31', freq='M')
    snowcover_file_df.loc[:, snowcover_file.columns] = snowcover_file
    snowcover_file_df.set_index("dates", inplace=True)
    snowcover_file_df.index.name = ""
    
    # Concatenate the DataFrames along the columns (axis=1)
    snowcover_df = pd.concat([snowcover_df, snowcover_file_df], axis=1)
    
snowcover_df

100%|██████████████████████████████████████████| 82/82 [00:00<00:00, 115.63it/s]


,FR004628,FR003492,FR003700,FR004933,FR004588,FR004501,FR004284,FR003231,FR003301,FR003382,...,FR003884,FR003956,FR003519,FR003683,FR003506,FR003230,FR003229,FR004660,FR003455,FR004324
2001-01-31,49.605632,1.595425,2.069738,0.888404,4.439499,1.914317,22.391146,2.638084,2.373946,4.0053,...,1.015054,2.365793,1.414729,1.486198,1.205101,3.810528,3.785976,14.730398,3.076086,2.173672
2001-02-28,44.260297,0.421377,1.081065,0.680808,3.877186,0.800296,6.187253,1.002283,0.968113,0.993214,...,0.29931,2.152325,0.122164,0.45915,0.645531,1.700602,1.689645,5.737472,0.621536,0.51512
2001-03-31,40.963719,0.628071,0.353354,0.629036,2.413304,0.661553,0.36624,0.514375,0.537787,2.312305,...,0.542685,0.078255,0.196805,0.625795,0.210518,0.865693,0.860113,3.300765,0.171954,1.256524
2001-04-30,38.406892,0.294598,0.575039,0.003125,0.106421,0.399618,0.181823,0.686027,0.651366,3.21137,...,2.048236,0.061661,0.964297,0.233628,0.198762,0.692351,0.688527,0.244492,0.417843,0.117552
2001-05-31,20.093963,0.103837,0.000711,0.001507,0.021479,0.008187,0.280215,0.002105,0.019305,0.016611,...,0.011188,0.003973,0.007159,0.0,0.005168,0.004721,0.004691,0.085899,0.010416,0.035205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,2.91808,0.022188,0.000116,0.0,0.00073,0.027721,0.157315,0.058289,0.011085,0.000861,...,0.004682,0.0,0.0,0.0,0.101876,0.0536,0.053254,0.0,0.0,0.001564
2022-09-30,3.918854,0.61362,1.169803,0.005909,0.152334,0.123596,0.010345,0.312312,0.451529,0.805868,...,0.036492,0.081635,0.556423,0.163041,0.24217,0.120047,0.119274,0.19699,0.116327,0.313194
2022-10-31,8.102375,0.021798,0.065633,0.009333,0.03101,0.02562,0.03157,0.012992,0.035186,0.013713,...,0.158944,0.053089,0.032954,0.127116,0.00115,0.005227,0.005193,0.04007,0.000601,0.001066
2022-11-30,26.552656,0.931084,1.645782,0.430983,2.826758,1.533492,0.354805,1.880222,1.530709,3.153003,...,1.871653,1.097486,0.604583,3.452291,0.738828,1.964895,1.952678,1.911979,1.585926,3.678584


In [7]:
# Here we add the columns of the catchemnts that were not processed
# Adding new columns with NaN values only if they don't exist
for col in catchment_boundaries.basin_id.tolist():
    if col not in snowcover_df.columns:
        snowcover_df[col] = np.nan
snowcover_df

,FR004628,FR003492,FR003700,FR004933,FR004588,FR004501,FR004284,FR003231,FR003301,FR003382,...,FR003884,FR003956,FR003519,FR003683,FR003506,FR003230,FR003229,FR004660,FR003455,FR004324
2001-01-31,49.605632,1.595425,2.069738,0.888404,4.439499,1.914317,22.391146,2.638084,2.373946,4.0053,...,1.015054,2.365793,1.414729,1.486198,1.205101,3.810528,3.785976,14.730398,3.076086,2.173672
2001-02-28,44.260297,0.421377,1.081065,0.680808,3.877186,0.800296,6.187253,1.002283,0.968113,0.993214,...,0.29931,2.152325,0.122164,0.45915,0.645531,1.700602,1.689645,5.737472,0.621536,0.51512
2001-03-31,40.963719,0.628071,0.353354,0.629036,2.413304,0.661553,0.36624,0.514375,0.537787,2.312305,...,0.542685,0.078255,0.196805,0.625795,0.210518,0.865693,0.860113,3.300765,0.171954,1.256524
2001-04-30,38.406892,0.294598,0.575039,0.003125,0.106421,0.399618,0.181823,0.686027,0.651366,3.21137,...,2.048236,0.061661,0.964297,0.233628,0.198762,0.692351,0.688527,0.244492,0.417843,0.117552
2001-05-31,20.093963,0.103837,0.000711,0.001507,0.021479,0.008187,0.280215,0.002105,0.019305,0.016611,...,0.011188,0.003973,0.007159,0.0,0.005168,0.004721,0.004691,0.085899,0.010416,0.035205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,2.91808,0.022188,0.000116,0.0,0.00073,0.027721,0.157315,0.058289,0.011085,0.000861,...,0.004682,0.0,0.0,0.0,0.101876,0.0536,0.053254,0.0,0.0,0.001564
2022-09-30,3.918854,0.61362,1.169803,0.005909,0.152334,0.123596,0.010345,0.312312,0.451529,0.805868,...,0.036492,0.081635,0.556423,0.163041,0.24217,0.120047,0.119274,0.19699,0.116327,0.313194
2022-10-31,8.102375,0.021798,0.065633,0.009333,0.03101,0.02562,0.03157,0.012992,0.035186,0.013713,...,0.158944,0.053089,0.032954,0.127116,0.00115,0.005227,0.005193,0.04007,0.000601,0.001066
2022-11-30,26.552656,0.931084,1.645782,0.430983,2.826758,1.533492,0.354805,1.880222,1.530709,3.153003,...,1.871653,1.097486,0.604583,3.452291,0.738828,1.964895,1.952678,1.911979,1.585926,3.678584


In [8]:
# Here we sort the columns:
snowcover_df = snowcover_df.sort_index(axis=1)

snowcover_df

,FR003159,FR003160,FR003161,FR003162,FR003163,FR003164,FR003165,FR003166,FR003167,FR003168,...,HR000308,HR000309,HR000310,HR000311,HR000312,HR000313,HR000314,HR000315,HR000316,HR000317
2001-01-31,3.802918,1.399932,0.0,1.039678,1.497816,4.767046,1.213882,1.666021,1.941474,1.812316,...,1.867276,0.595316,5.382383,2.842625,3.489033,5.638406,9.571026,3.301846,2.939173,10.381366
2001-02-28,0.518688,0.3395,0.017037,0.209031,0.326979,0.13246,0.331365,0.429403,0.352797,0.318855,...,2.022155,0.332684,3.208757,1.735772,2.063006,3.367068,5.602633,1.957926,1.761288,3.02557
2001-03-31,1.189231,0.585266,0.0,1.220122,0.679022,0.004836,0.411743,0.377319,0.2019,0.429508,...,0.136005,1.956961,0.141307,0.084849,0.089908,0.148327,0.242281,0.087113,0.084667,1.53047
2001-04-30,0.574412,0.635984,0.241892,1.090995,0.990512,0.027088,0.661938,0.655891,0.444778,0.722544,...,0.0,0.0,0.193059,0.071364,0.11547,0.20265,0.327397,0.109275,0.097272,0.830166
2001-05-31,0.0,0.000465,0.0,0.0,0.00032,0.0,0.004576,0.00359,0.013216,0.005934,...,0.013331,0.000417,0.002458,0.000909,0.00147,0.00258,0.004694,0.001391,0.001238,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.0,0.0,0.0,0.0,0.0,0.0,0.004086,0.003205,0.017767,0.007763,...,0.0,0.0,0.001156,0.000427,0.000692,0.001214,0.002208,0.000655,0.000583,0.001316
2022-09-30,0.012937,0.050039,0.0,0.209093,0.078824,0.0,0.5395,0.423262,0.192114,0.123331,...,0.351022,1.482715,0.250767,0.207739,0.179279,0.263224,0.472807,0.181269,0.161623,0.917728
2022-10-31,0.0,0.000744,0.0,0.0,0.000511,0.0,0.054755,0.042957,0.019034,0.008572,...,0.009668,0.0,0.001887,0.018837,0.004067,0.001981,0.003604,0.01484,0.013425,0.049186
2022-11-30,2.063833,2.297022,0.34034,3.87427,2.736351,1.450599,2.552961,2.824091,2.908334,2.873194,...,0.354663,0.0,1.731858,1.29333,1.412794,1.817754,2.57698,1.498851,1.574832,3.014434


In [9]:
# Resample to yearly mean
snowcover_yr = snowcover_df.resample('Y').mean()

snowcover_yr

,FR003159,FR003160,FR003161,FR003162,FR003163,FR003164,FR003165,FR003166,FR003167,FR003168,...,HR000308,HR000309,HR000310,HR000311,HR000312,HR000313,HR000314,HR000315,HR000316,HR000317
2001-12-31,2.446204,1.849634,2.220707,2.021921,1.881161,2.169912,1.689269,1.715,1.5154,1.707944,...,3.55753,3.363196,2.38801,1.170819,1.52824,2.502762,4.103643,1.462382,1.330466,2.48281
2002-12-31,3.280235,2.488846,3.260003,1.680424,2.163279,3.688894,3.170272,3.046664,1.975398,1.975139,...,4.311676,2.881381,3.170362,1.584038,2.158938,3.288342,4.591872,2.056105,1.914516,2.605617
2003-12-31,3.818328,2.219731,2.260413,1.668025,1.971672,4.78473,2.284752,2.126976,1.792653,1.832947,...,7.199595,3.649559,7.172757,3.647065,4.749944,7.506795,10.571361,4.529973,4.124719,4.29121
2004-12-31,5.803956,5.785545,5.941914,6.324264,5.879935,5.990071,5.742416,5.757521,5.281838,5.51371,...,3.598778,2.059531,6.853763,3.373073,4.512957,7.161044,10.946556,4.280025,3.869806,4.82832
2005-12-31,10.498502,6.873512,6.732795,5.864856,6.392201,11.576529,7.707975,6.925261,4.872206,5.525018,...,9.50902,7.749252,9.199945,4.530019,6.090294,9.612134,14.203738,5.771068,5.208135,4.732844
2006-12-31,5.842811,3.445759,2.98135,2.791903,3.146057,5.849362,4.455835,4.024906,2.862496,2.936028,...,1.424537,2.243458,2.783024,1.239451,1.728857,2.91461,4.639681,1.64277,1.476524,5.810957
2007-12-31,3.588756,2.067897,2.051907,2.484125,2.182457,5.619704,3.0396,2.818057,2.730962,2.451493,...,1.090205,0.873698,2.834269,1.328824,1.808935,2.966828,4.757359,1.727954,1.563238,3.323308
2008-12-31,1.610298,0.868795,1.237425,1.130306,0.941018,2.621864,1.228896,1.064812,1.132862,1.063665,...,3.094772,1.657128,1.095685,0.605711,0.742134,1.141166,1.658314,0.713964,0.680422,3.213887
2009-12-31,5.865621,3.253271,3.956169,2.295406,2.850141,6.186719,3.708106,3.091776,2.574077,2.706963,...,3.30419,3.066952,5.655662,3.086018,3.86357,5.906851,8.710117,3.708699,3.477606,3.666771
2010-12-31,9.991942,9.157999,10.111768,9.495695,9.144489,11.774504,9.878225,9.65205,8.849841,8.916543,...,8.275084,4.718481,6.403621,3.235539,4.140487,6.707519,10.597094,3.933022,3.579891,6.909236


In [10]:
# Calculate the mean for each month across all years (monht of the year)
snowcover_moy = snowcover_df.groupby(snowcover_df.index.month).mean()

# Rename the index to the three-letter month abbreviations
snowcover_moy.index = pd.to_datetime(snowcover_moy.index, format='%m').strftime('%b')

snowcover_moy

,FR003159,FR003160,FR003161,FR003162,FR003163,FR003164,FR003165,FR003166,FR003167,FR003168,...,HR000308,HR000309,HR000310,HR000311,HR000312,HR000313,HR000314,HR000315,HR000316,HR000317
Jan,17.535639,13.072658,15.965256,12.949485,12.680375,21.277439,15.790424,15.023577,12.533545,12.394361,...,14.72326,11.234305,16.147373,8.113623,10.605953,16.853545,24.999318,10.096491,9.234403,11.279077
Feb,11.611134,7.694843,8.796972,7.248245,7.300072,13.675413,9.134016,8.453126,6.331635,6.714506,...,11.4748,5.765303,14.825639,6.941576,9.575606,15.510863,24.042661,9.082481,8.205285,11.700072
Mar,4.311543,2.296816,1.658547,1.376322,1.954439,5.309741,3.187028,2.762259,1.581544,1.703461,...,2.732174,2.625626,6.870963,3.343227,4.485103,7.18307,10.966783,4.253701,3.867658,7.294893
Apr,0.842069,0.489088,0.434016,0.315079,0.424226,0.905174,0.682553,0.609478,0.362337,0.373936,...,0.018644,0.133211,0.504612,0.23732,0.334753,0.514952,0.79271,0.318308,0.292527,1.668658
May,0.036994,0.053698,0.019825,0.061309,0.057089,0.167394,0.139082,0.140853,0.158972,0.10046,...,0.021501,0.013582,0.049487,0.063743,0.082601,0.051078,0.054714,0.078617,0.074825,0.124052
Jun,0.138218,0.05467,0.004918,0.010081,0.039783,0.258317,0.100869,0.094373,0.065909,0.048681,...,0.011873,0.024396,0.094558,0.053448,0.061933,0.095666,0.10956,0.059496,0.061004,0.041014
Jul,0.078285,0.053149,0.004296,0.010494,0.039645,0.004275,0.047325,0.038167,0.022037,0.029465,...,0.005897,0.00844,0.041904,0.054924,0.045542,0.040702,0.02814,0.046318,0.049014,0.060501
Aug,0.03069,0.04804,0.039252,0.023893,0.04498,0.198539,0.131738,0.110482,0.07404,0.055385,...,0.011228,0.024225,0.047109,0.03501,0.037264,0.048478,0.045904,0.038511,0.034774,0.039262
Sep,0.06085,0.083907,0.080635,0.081557,0.077096,0.015753,0.088869,0.100457,0.103358,0.089664,...,0.031893,0.070261,0.087797,0.095899,0.091246,0.085643,0.049153,0.091997,0.088847,0.255143
Oct,0.580556,0.291387,0.054349,0.178154,0.24862,1.071796,0.388016,0.345608,0.23991,0.232865,...,0.050576,0.104075,0.320672,0.197991,0.235715,0.333755,0.467407,0.228866,0.217186,0.381531


# Final aggregation (static attributes)

In [11]:
# Snow cover:
snowcover_moy_T = snowcover_moy.T
snowcover_moy_T.columns = pd.to_datetime(snowcover_moy_T.columns, format='%b').strftime('%m')
snowcover_moy_T.columns = "sno_cov_" + snowcover_moy_T.columns
snowcover_moy_T["sno_cov_mean"] = snowcover_moy_T.mean(axis = 1)
snowcover_moy_T

,sno_cov_01,sno_cov_02,sno_cov_03,sno_cov_04,sno_cov_05,sno_cov_06,sno_cov_07,sno_cov_08,sno_cov_09,sno_cov_10,sno_cov_11,sno_cov_12,sno_cov_mean
FR003159,17.535639,11.611134,4.311543,0.842069,0.036994,0.138218,0.078285,0.03069,0.06085,0.580556,2.483944,14.560012,4.355828
FR003160,13.072658,7.694843,2.296816,0.489088,0.053698,0.05467,0.053149,0.04804,0.083907,0.291387,2.064659,10.830579,3.086124
FR003161,15.965256,8.796972,1.658547,0.434016,0.019825,0.004918,0.004296,0.039252,0.080635,0.054349,1.915277,10.768313,3.311805
FR003162,12.949485,7.248245,1.376322,0.315079,0.061309,0.010081,0.010494,0.023893,0.081557,0.178154,1.882861,10.634444,2.89766
FR003163,12.680375,7.300072,1.954439,0.424226,0.057089,0.039783,0.039645,0.04498,0.077096,0.24862,1.974035,10.625602,2.955497
...,...,...,...,...,...,...,...,...,...,...,...,...,...
HR000313,16.853545,15.510863,7.18307,0.514952,0.051078,0.095666,0.040702,0.048478,0.085643,0.333755,1.892565,9.023195,4.302793
HR000314,24.999318,24.042661,10.966783,0.79271,0.054714,0.10956,0.02814,0.045904,0.049153,0.467407,2.861542,13.863236,6.523427
HR000315,10.096491,9.082481,4.253701,0.318308,0.078617,0.059496,0.046318,0.038511,0.091997,0.228866,1.278665,5.504449,2.589825
HR000316,9.234403,8.205285,3.867658,0.292527,0.074825,0.061004,0.049014,0.034774,0.088847,0.217186,1.214453,5.162866,2.375237


In [12]:
# Assign the "basin_id" to the gauges names:
snowcover_moy_T.index.name = "basin_id"

In [13]:
# Assign the "date" as the index name of the df:
snowcover_yr.index.name = "date"
snowcover_df.index.name = "date"

In [14]:
# Round the data to 3 decimals
snowcover_df = snowcover_df.astype(float).round(3)
snowcover_yr = snowcover_yr.astype(float).round(3)
snowcover_moy_T = snowcover_moy_T.astype(float).round(3)

# Data export

In [15]:
# Export the final datasets:
# Time-series:
snowcover_df.to_csv(PATH_OUTPUT_TS+"/estreams_snowcover_monhtly.csv")
snowcover_yr.to_csv(PATH_OUTPUT_TS+"/estreams_snowcover_yearly.csv")

# Static attributes:
snowcover_moy_T.to_csv(PATH_OUTPUT_ST+"/estreams_snowcover_attributes.csv")

# End